# Titre 1 : Audit via données randomisées 

In [1]:
# STEP 1 — Setup (10Y version)
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

ticker = "AAPL"
tkr = yf.Ticker(ticker)

# ✅ Quarterly statements (gives a longer history than annual)
income = tkr.quarterly_financials
bs = tkr.quarterly_balance_sheet
cf = tkr.quarterly_cashflow

print("✅ Loaded yfinance QUARTERLY statements for:", ticker)
print("Income shape:", income.shape)
print("Balance sheet shape:", bs.shape)
print("Cashflow shape:", cf.shape)


/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/fundamentals.py:120: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  end = pd.Timestamp.utcnow().ceil("D")
/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/fundamentals.py:120: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  end = pd.Timestamp.utcnow().ceil("D")
/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/fundamentals.py:120: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  end = pd.Timestamp.utcnow().ceil("D")


✅ Loaded yfinance QUARTERLY statements for: AAPL
Income shape: (33, 5)
Balance sheet shape: (65, 6)
Cashflow shape: (46, 7)


In [2]:
# STEP 2 — Inspect available row names (10Y / quarterly)
# (same idea, just keep it clean + show how many rows/cols you have)

print("=== INCOME STATEMENT rows (first 40) ===")
print(income.index.tolist()[:40])
print("Columns (dates):", list(income.columns[:8]), "...")

print("\n=== BALANCE SHEET rows (first 40) ===")
print(bs.index.tolist()[:40])
print("Columns (dates):", list(bs.columns[:8]), "...")

print("\n=== CASHFLOW rows (first 40) ===")
print(cf.index.tolist()[:40])
print("Columns (dates):", list(cf.columns[:8]), "...")


=== INCOME STATEMENT rows (first 40) ===
['Tax Effect Of Unusual Items', 'Tax Rate For Calcs', 'Normalized EBITDA', 'Net Income From Continuing Operation Net Minority Interest', 'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA', 'EBIT', 'Normalized Income', 'Net Income From Continuing And Discontinued Operation', 'Total Expenses', 'Total Operating Income As Reported', 'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders', 'Net Income Common Stockholders', 'Net Income', 'Net Income Including Noncontrolling Interests', 'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income', 'Other Income Expense', 'Other Non Operating Income Expenses', 'Operating Income', 'Operating Expense', 'Research And Development', 'Selling General And Administration', 'Gross Profit', 'Cost Of Revenue', 'Total Revenue', 'Operating Revenue']
Columns (dates): [Timestamp('2025-12-31 00:00:00'), Timestamp('2025-09-30 00:00:00'), 

In [5]:
# STEP 3 FIX — You don't have 10Y because yfinance quarterly fundamentals are LIMITED.
# Let's CHECK how many quarters you actually received, then build "as much history as available".

print("Income quarters:", len(income.columns))
print("Balance sheet quarters:", len(bs.columns))
print("Cashflow quarters:", len(cf.columns))

print("\nIncome date range:", min(income.columns), "→", max(income.columns))
print("BS date range:", min(bs.columns), "→", max(bs.columns))
print("CF date range:", min(cf.columns), "→", max(cf.columns))


Income quarters: 5
Balance sheet quarters: 6
Cashflow quarters: 7

Income date range: 2024-12-31 00:00:00 → 2025-12-31 00:00:00
BS date range: 2024-09-30 00:00:00 → 2025-12-31 00:00:00
CF date range: 2024-06-30 00:00:00 → 2025-12-31 00:00:00


In [6]:
# STEP 3 (robust) — build annual from whatever quarters exist (no forced 10Y)

def pick_q(df, row_name, col_name):
    s = df.loc[row_name]
    s.index = pd.to_datetime(s.index)
    s = pd.to_numeric(s, errors="coerce").sort_index()
    s.name = col_name
    return s

def fiscal_year_from_date(dt):
    return dt.year + (1 if dt.month >= 10 else 0)

q = pd.concat([
    pick_q(income, "Total Revenue", "revenue"),
    pick_q(income, "Net Income", "net_income"),
    pick_q(income, "EBIT", "ebit"),
    pick_q(income, "Diluted Average Shares", "diluted_shares"),

    pick_q(bs, "Total Assets", "assets"),
    pick_q(bs, "Stockholders Equity", "equity"),
    pick_q(bs, "Total Debt", "total_debt"),

    pick_q(cf, "Free Cash Flow", "fcf"),
    pick_q(cf, "Repurchase Of Capital Stock", "buybacks"),
    pick_q(cf, "Capital Expenditure", "capex"),
    pick_q(cf, "Cash Dividends Paid", "dividends_paid"),
], axis=1)

q["fiscal_year"] = [fiscal_year_from_date(d) for d in q.index]

flow_cols = ["revenue","net_income","ebit","fcf","buybacks","capex","dividends_paid"]
stock_cols = ["assets","equity","total_debt","diluted_shares"]

annual_flow = q.groupby("fiscal_year")[flow_cols].sum(min_count=1)
annual_stock = q.groupby("fiscal_year")[stock_cols].last()

data = annual_flow.join(annual_stock, how="inner").sort_index()

# drop any fully-empty fiscal year (like your 2024 NaN row)
data = data.dropna(subset=["revenue","net_income","assets","equity"], how="any")

# derived metrics
data["net_margin"] = data["net_income"] / data["revenue"]
data["roe"] = data["net_income"] / data["equity"]
data["leverage"] = data["assets"] / data["equity"]
data["eps"] = data["net_income"] / data["diluted_shares"]
data["buybacks_to_fcf"] = data["buybacks"] / data["fcf"]
data["asset_turnover"] = data["revenue"] / data["assets"]

print("✅ Fiscal years available:", data.index.tolist())
data


✅ Fiscal years available: [2025, 2026]


/tmp/ipykernel_42882/2629448289.py:13: Pandas4Warning: Sorting by default when concatenating all DatetimeIndex is deprecated.  In the future, pandas will respect the default of `sort=False`. Specify `sort=True` or `sort=False` to silence this message. If you see this warnings when not directly calling concat, report a bug to pandas.
  q = pd.concat([


,revenue,net_income,ebit,fcf,buybacks,capex,dividends_paid,assets,equity,total_debt,diluted_shares,net_margin,roe,leverage,eps,buybacks_to_fcf,asset_turnover
fiscal_year,,,,,,,,,,,,,,,,,
2025,4.161610e+11,1.120100e+11,1.330500e+11,9.876700e+10,-9.071100e+10,-1.271500e+10,-1.542100e+10,3.592410e+11,7.373300e+10,9.865700e+10,1.486361e+10,0.269151,1.519130,4.872187,7.535855,-0.918434,1.158445
2026,1.437560e+11,4.209700e+10,5.085200e+10,5.155200e+10,-2.470100e+10,-2.373000e+09,-3.921000e+09,3.792970e+11,8.819000e+10,9.050900e+10,1.481036e+10,0.292836,0.477344,4.300907,2.842403,-0.479147,0.379006


In [1]:
! pip install matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 75.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [seaborn]m7/8 [seaborn]ib]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
#première cellule : est la cellule des impprts de packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt #ggplot2 en R
import seaborn as sns


In [3]:
#cell 2 is about configuration
sns.set_theme(style="whitegrid")

In [ ]:
#pour l'exercice d'un dataset
#on utilise NUMPY